<a href="https://colab.research.google.com/github/AdrianDiazGaxiola/Proyecto-Final/blob/main/Sequ%C3%ADas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este programa se empleará para limpiar el DataFrame con los datos de las sequías en México

In [10]:
!pip install numpy
!pip install pandas
!pip install openpyxl

Se copia el archivo de github con los datos a un dataframe

In [12]:
import pandas as pd

url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/blob/OtherData/Municipios%20con%20Sequ%C3%ADa.csv?raw=true"
Seq = pd.read_csv(url)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (92,93,131,132,301) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
seq_por_entidad = {}



      Clave del Municipio   Nombre del Municipio  ... D4 (D4) Total (D0 a D4)
0                  1001.0         Aguascalientes  ...     1.0            65.0
1                  1002.0               Asientos  ...     0.0            64.0
2                  1003.0               Calvillo  ...     2.0            71.0
3                  1004.0                  Cosio  ...     0.0            61.0
4                  1005.0            Jesús Maria  ...     2.0            66.0
...                   ...                    ...  ...     ...             ...
2460              32056.0              Zacatecas  ...     0.0            75.0
2461              32057.0               Trancoso  ...     0.0            73.0
2462              32058.0  Santa Maria de la Paz  ...     3.0           121.0
2463                  NaN                    NaN  ...     NaN             NaN
2464                  NaN                    NaN  ...     NaN             NaN

[2465 rows x 307 columns]
